# Siemens Logistics - Join Tracking with Shipments

This notebook aims to joins the shipments descriptions with the tracking event records and format the resulting dataset so that it can be used to train the AI agents.

## Imports

### Imports

In [166]:
# Imports
# Create_Flat_Tracking
import pandas as pd
import numpy as np
import os
import json
import glob
from datetime import datetime
from tqdm.autonotebook import tqdm
tqdm.pandas()

### Mount Google Drive

In [167]:

# Mount my Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import shipment Data

In [168]:
#  initialize and import the shipment dataframe
df_shipments = pd.read_json("/content/drive/MyDrive/Colab_Notebooks/Data/result/shipment_export_16_12_2020.json", orient='records')

# list statistics of dataframe
# Statistics
print("### Shipments Data set statistics")
print("## Total Number of shipments: ", len(df_shipments.index))
df_shipments.head()

### Shipments Data set statistics
## Total Number of shipments:  262981


,SHIPMENTID,Shipment_Distance,Qty__INT,Volume__FLOAT,Weight__FLOAT,Logistics_Provider,Sender,Receiver,Sender_City,Sender_Country,Sender_City_Lat,Sender_City_Long,Receiver_City,Receiver_Country,Receiver_City_Lat,Receiver_City_Long,Creator
0,297941628,134.444124,5,0,770,Heppner GmbH & Co. KG - KEHL / RHEIN,EVOCA Germany GmbH,Heuser GmbH,Rastatt,DE,48.85851,8.20965,Dietzenbach,DE,50.00976,8.77783,Vollmer
1,297956359,471.992586,1,0,119,Heppner GmbH & Co. KG - KORNWESTHEIM,U.I. LAPP GMBH,ELECTRO RHONE VENISSIEUX,STUTTGART,DE,48.78232,9.17702,Venissieux,FR,45.70254,4.87147,EDI
2,297956366,471.992586,1,0,67,Heppner GmbH & Co. KG - KORNWESTHEIM,U.I. LAPP GMBH,ELECTRO RHONE VENISSIEUX,STUTTGART,DE,48.78232,9.17702,Venissieux,FR,45.70254,4.87147,EDI
3,297956365,471.992586,1,0,244,Heppner GmbH & Co. KG - KORNWESTHEIM,U.I. LAPP GMBH,ELECTRO RHONE VENISSIEUX,STUTTGART,DE,48.78232,9.17702,Venissieux,FR,45.70254,4.87147,EDI
4,297956363,471.992586,1,0,296,Heppner GmbH & Co. KG - KORNWESTHEIM,U.I. LAPP GMBH,ELECTRO RHONE VENISSIEUX,STUTTGART,DE,48.78232,9.17702,Venissieux,FR,45.70254,4.87147,EDI


### Import Tracking data

In [169]:
# Initialize and import the tacking dataframe
df_tracking = pd.read_json("/content/drive/MyDrive/Colab_Notebooks/Data/results/tracking_status_processed_28_01_2021_0_200000.json", orient='records')

# formt columbs
df_tracking[[c for c in df_tracking.columns if c.endswith('_DATE')]] = \
    df_tracking[[c for c in df_tracking.columns if  c.endswith('_DATE')]].progress_apply(lambda x:pd.to_datetime(x,  unit='ms'))

# list statistics of dataframe
# Statistics
print("### Tracking Data set statistics")
print("## Total Number of events/records: ", len(df_tracking.index))
print("## Total Number of distincts events/records: ", len(df_tracking.drop_duplicates(['SHIPMENTID', 'reason_DE', 'status_DE']).index))



### Tracking Data set statistics
## Total Number of events/records:  200000
## Total Number of distincts events/records:  200000


## Preprocess Tracking Data

In [170]:
# Sort the tracking by "SHIPMENT_ID" and "StatusDate"
df_tracking = df_tracking.sort_values(['SHIPMENTID', 'STATUS_DATE'])


#### Duration of all events

In [171]:
# Compute the duration of all events
# Group and compute the time difference between events
df_tracking['EVENT_DURATION'] = df_tracking.groupby(['SHIPMENTID'], sort=False)["STATUS_DATE"].diff(periods= -1).fillna(pd.Timedelta(seconds=0))
df_tracking['EVENT_DURATION'] = df_tracking['EVENT_DURATION'].abs()
df_tracking['EVENT_DURATION_SECONDS'] = df_tracking['EVENT_DURATION'].progress_apply(lambda x: x.total_seconds())
df_tracking.loc[df_tracking['EVENT_DURATION'] > pd.Timedelta(seconds=0)].head()

,LOCATION,reason_DE,END_DATE,START_DATE,STATUS_DATE,status_EN,EXPLANATION,SYSTEM_DATE,OBJECT_ID,status_DE,SHIPMENTID,TS_ID,reason_EN,MILESTONES,REIHENFOLGE,MILESTONES_START_DATE,MILESTONES_EVENT_COUNT,MILESTONES_END_DATE,EVENT_DURATION,EVENT_DURATION_SECONDS
4,EDI,Zustellung nicht belegbar/Wechselbrückengestel...,2019-08-01 05:00:00,2019-01-01 05:00:00,2019-01-03 10:06:00,Delivered,,2019-01-03 10:23:30,-221888158,Zugestellt,295476310,2007762918,delivery without POD / swap body deposit,Zustellung,600,2019-01-03 10:06:00,2,2019-01-25 10:47:00,22 days 00:41:00,1903260.0
51,EDI,Zustellung nicht belegbar/Wechselbrückengestel...,2019-01-15 05:00:00,2019-08-01 05:00:00,2019-01-11 07:29:00,Delivered,,2019-01-11 07:38:19,-187265006,Zugestellt,295476311,2022267746,delivery without POD / swap body deposit,Zustellung,600,2019-01-11 07:29:00,2,2019-01-25 10:47:00,14 days 03:18:00,1221480.0
3,EDI,Sendung auf dem Weg zum Empfänger/in Zustellung,2019-08-01 05:00:00,2019-01-01 05:00:00,2019-01-03 00:00:00,delivery,,2019-01-03 13:53:18,-542459960,Zustellung,297166330,2008041575,En route to consignee,In Zustellung,500,2019-01-03 00:00:00,1,2019-02-05 23:59:00,33 days 23:59:00,2937540.0
68,EDI,Eingang Versandpartner,2019-01-15 05:00:00,2019-08-01 05:00:00,2019-01-14 16:06:00,discharge,NEU-ULM 5082,2019-01-14 16:23:27,1151992755,Entladung,297200938,2026896715,Inbound Sending Partner,Ankunft Zustellterminal,490,2019-01-14 16:06:00,1,2020-01-13 23:57:00,1 days 01:14:00,90840.0
77,EDI,Verladen an,2019-01-22 05:00:00,2019-01-15 05:00:00,2019-01-15 17:20:00,dispatch,Rungis (5594),2019-01-15 17:53:25,-941654826,Abfertigung,297200938,2029706018,loaded to,Abfahrt von Abholterminal,410,2019-01-15 17:20:00,1,2019-01-14 16:06:00,1 days 17:15:00,148500.0


#### Package that have been delivered

In [172]:
# Select Data with milestone "Zustellung" or archiviert
df_events_with_zugeliefert = df_tracking[df_tracking['REIHENFOLGE'] == 600]
df_uniques_shipments_zugeliefert = df_events_with_zugeliefert['SHIPMENTID'].unique()

# assign the label
df_tracking["DELIVERED"] = 0
df_tracking["DELIVERED"] = np.where(df_tracking.isin(df_uniques_shipments_zugeliefert), 0, 1)
# df_tracking.loc[df_tracking['EVENT_DURATION'] > pd.Timedelta(seconds=0)].head()s

#### Total Delivery Duration

In [173]:
df_tracking["DELIVERY_DURATION"] = 31622400 # Number of seconds in a year
df_tmp = df_tracking[(df_tracking['REIHENFOLGE'] <= 600) & (df_tracking["DELIVERED"] == 1)]
df_tmp["DELIVERY_DURATION"] = df_tmp.groupby(['SHIPMENTID'], sort=False)["EVENT_DURATION_SECONDS"].transform('sum')
df_tracking["DELIVERY_DURATION"] = df_tmp["DELIVERY_DURATION"]
df_tracking["DELIVERY_DURATION"].fillna(31622400, inplace=True)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [174]:
df_tracking.loc[df_tracking['MILESTONES'] == 'Archiviert']

,LOCATION,reason_DE,END_DATE,START_DATE,STATUS_DATE,status_EN,EXPLANATION,SYSTEM_DATE,OBJECT_ID,status_DE,SHIPMENTID,TS_ID,reason_EN,MILESTONES,REIHENFOLGE,MILESTONES_START_DATE,MILESTONES_EVENT_COUNT,MILESTONES_END_DATE,EVENT_DURATION,EVENT_DURATION_SECONDS,DELIVERED,DELIVERY_DURATION
355,EDI,Ablieferbeleg (POD) archiviert (FR),2019-12-02 05:00:00,2019-05-02 05:00:00,2019-02-05 23:59:00,archived,,2019-02-06 06:23:18,874347838,Archiviert,297166330,2072795291,POD archived,Archiviert,650,2019-02-05 23:59:00,1,2019-02-05 23:59:00,0 days,0.0,1,31622400.0
49,EDI,Ablieferbeleg (POD) archiviert (FR),2019-01-15 05:00:00,2019-08-01 05:00:00,2019-01-10 23:59:00,archived,,2019-01-11 06:23:32,-471708799,Archiviert,297346414,2022130118,POD archived,Archiviert,650,2019-01-10 23:59:00,2,2019-08-05 13:04:00,0 days,0.0,1,31622400.0
58,EDI,Ablieferbeleg (POD) archiviert (FR),2019-01-15 05:00:00,2019-08-01 05:00:00,2019-01-11 23:59:00,archived,,2019-01-14 06:24:21,-1593020296,Archiviert,298368425,2026039241,POD archived,Archiviert,650,2019-01-11 23:59:00,1,2019-01-11 23:59:00,0 days,0.0,1,31622400.0
175,EDI,Ablieferbeleg (POD) archiviert (FR),2019-01-29 05:00:00,2019-01-22 05:00:00,2019-01-23 23:59:00,archived,,2019-01-24 06:23:11,1946891336,Archiviert,299016319,2047712501,POD archived,Archiviert,650,2019-01-23 23:59:00,2,2019-11-12 13:45:00,0 days,0.0,1,31622400.0
147,EDI,Ablieferbeleg (POD) archiviert (FR),2019-01-29 05:00:00,2019-01-22 05:00:00,2019-01-22 23:59:00,archived,,2019-01-23 06:23:23,759368259,Archiviert,299725987,2045196585,POD archived,Archiviert,650,2019-01-22 23:59:00,1,2019-01-22 23:59:00,0 days,0.0,1,31622400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42875,EDI,Ablieferbeleg (POD) archiviert (FR),2019-12-03 05:00:00,2019-05-03 05:00:00,2019-03-09 23:59:00,archived,,2020-03-09 15:38:18,593547626,Archiviert,342745469,2705419761,POD archived,Archiviert,650,2019-03-09 23:59:00,1,2019-03-09 23:59:00,0 days,0.0,1,31622400.0
42874,EDI,Ablieferbeleg (POD) archiviert (FR),2019-12-03 05:00:00,2019-05-03 05:00:00,2019-03-09 23:59:00,archived,,2020-03-09 15:38:18,915790857,Archiviert,342815643,2705419766,POD archived,Archiviert,650,2019-03-09 23:59:00,1,2019-03-09 23:59:00,0 days,0.0,1,31622400.0
42877,EDI,Ablieferbeleg (POD) archiviert (FR),2019-12-03 05:00:00,2019-05-03 05:00:00,2019-03-09 23:59:00,archived,,2020-03-09 15:38:18,1140567632,Archiviert,343001450,2705419778,POD archived,Archiviert,650,2019-03-09 23:59:00,1,2019-03-09 23:59:00,0 days,0.0,1,31622400.0
42876,EDI,Ablieferbeleg (POD) archiviert (FR),2019-12-03 05:00:00,2019-05-03 05:00:00,2019-03-09 23:59:00,archived,,2020-03-09 15:38:18,-1053331178,Archiviert,343047633,2705419779,POD archived,Archiviert,650,2019-03-09 23:59:00,1,2019-03-09 23:59:00,0 days,0.0,1,31622400.0


## Join Status with shipment data

In [175]:
df_merged = pd.merge(df_tracking, df_shipments, on='SHIPMENTID')
print(len(df_merged.index))

199971


## Export All

In [176]:
df_merged.to_json(r'/content/drive/MyDrive/Colab_Notebooks/Data/results/data_merged_export_06_02_2021.json', orient='records')